# Explain the concept of tidy data
Reference paper: [Tidy Data](https://vita.had.co.nz/papers/tidy-data.pdf)
Reference DataCamp course: Reshaping data with pandas

Definition: 

In tidy data:
- Each variable forms a column
- Each observation forms a row
- Each type of observational unit forms a table

# Fundamental operations
There are three fundamental operations:
- Pivot
- Stack / Unstack
- Melt

Reference: https://pandas.pydata.org/docs/user_guide/reshaping.html

Reference: https://wesmckinney.com/book/data-wrangling.html#prep_reshape